# Лабораторная работа 3
## Фильтр частиц
### Описание системы

В квадратной площадке длиной 100 метров находится мобильный робот. В углах площадки установлены радиомаяки. Робот может измерять расстояние до маяков. Погрешность измерений описывается нормальным распределением с нулевым математически ожиданием и погрешностью $\sigma = 15$. Известно, что начальное положение робота находится в диапазоне координат (0 -100, 0- 100). Начальная ориентация робота составляет $0 \pm 2$ градуса.

Робот функционирует следующим образом:

- Принимает от оператора требуемый угол поворота и дальность перемещения;
- Поворачивается на заданный угол;
- Проезжает заданную дистанцию;
- Измеряет расстояние до маяков;
- Рассчитывает свое положение.

Поворот робота на заданный угол происходит с нормально распределенной погрешностью с нулевым математическим ожиданием и средневадратическим отклонением $\sigma_{\theta} = 2$. Перемещение робота на заданную дистанцию происходит с нормально распределенной погрешностью с нулевым математическим ожиданием и средневадратическим отклонением $\sigma_{l} = 15$.  Требуется построить фильтр частиц для повышения точности локализации робота.

### Задание

- Задать начальное положение робота и средневадратические отклонения;
- Задать не менее 5 команд на перемещение робота;
- Построить траеткторию движения робота с учетом погрешностей;
- Построить фильтр частиц для робота;
- Построить траеткорию движения робота, положения частиц и оценку положения робота после выполнения каждой из команд;
- Выполнить предыдущие пункты для трех различных начальных положений, среднеквадратических отклонений, количества частиц и траекторий движения.

### 1. Задать начальное положение робота и среднеквадратические отклонения

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [10]:
# Класс робота

class Robot:
    def __init__(self, x, y, ang, sigma_move, sigma_turn, sigma_meas) -> None:
        # Истинное положение робота и угол
        self._true_pos = np.array([[x], 
                                   [y]])
        self._calc_pos = np.array([[0], 
                                   [0]])
        self._ang = ang
        # среднеквадратическое отклонение при перемещении
        self._sigma_move = sigma_move
        # среднеквадратическое отклонение при повороте
        self._sigma_turn = sigma_turn
        
        # Измерение расстояния до маяка
        self._beacon_meas = np.array([[0],  # 1 маяк
                                      [0],  # 2 маяк
                                      [0],  # 3 маяк
                                      [0]]) # 4 маяк
        # среднеквадратическое отклонение при измерении
        self._sigma_meas = sigma_meas
        
        
    def move(self, path_len, ang):
        self._ang += ang + np.random.normal(0, self._sigma_turn)
        self._true_pos[0] += path_len * np.cos(self._ang) + np.random.normal(0, self._sigma_move)
        self._true_pos[1] += path_len * np.sin(self._ang) + np.random.normal(0, self._sigma_move)
        
        return self._true_pos
    
    
    def make_measurment(self, beacons):
        #  b3(0,y3)*       * b4 (x4, y4)    # Расположегие маяков
        #                                   #
        #                                   #
        #  b1(0,0) *       * b2 (x2, 0)     #
        meas = np.zeros([4,1])
        for i in range(len(meas)):
            meas[i] = np.sqrt((beacons[i][0] - self._true_pos[0])**2 + (beacons[i][1] - self._true_pos[1])**2) + np.random.normal(0, self._sigma_meas)
        self._beacon_meas = meas
        
        return self._beacon_meas
    
    
    def calc_pos(self, beacons):
        # Вычисление положение при помощи трилтерации по двум маякам
        x_calc = (self._beacon_meas[0]**2 - self._beacon_meas[1]**2 + beacons[1][0]**2)/(2*beacons[1][0])
        y_calc = (self._beacon_meas[0]**2 - self._beacon_meas[3]**2 + beacons[2][1]**2)/(2*beacons[2][1])
        self._calc_pos = np.array([[x_calc],
                                   [y_calc]])
        
        
    def get_true_pos(self):
        return self._true_pos, self._ang
    
    
    def get_calc_pos(self):
        return self._calc_pos, self._ang

In [8]:
kek = np.zeros([2,1])